In [9]:
import mxnet as mx
from mxnet import nd
from mxnet.gluon import nn

# Multilayer perceptrons
class MLP(nn.Block):
    # 声明带有模型参数的层，这里声明了两个全连接层
    def __init__(self, **kwargs):
        # 调用MLP父类Block的构造函数来进行必要的初始化。这样在构造实例时还可以指定其他函数
        # 参数，如“模型参数的访问、初始化和共享”一节将介绍的模型参数params
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Dense(256, activation='relu')  # 隐藏层
        self.output = nn.Dense(10)  # 输出层

    # 定义模型的前向计算，即如何根据输入x计算返回所需要的模型输出
    # MLP类中无须定义反向传播函数。系统将通过自动求梯度而自动生成反向传播所需的backward函数。
    def forward(self, x):
        return self.output(self.hidden(x))

In [10]:
X = nd.random.uniform(shape=(2, 20),ctx=mx.gpu())
net = MLP()
net.initialize(ctx=mx.gpu())
net(X)


[[-0.00711333  0.00847135  0.00836362  0.00435072  0.01855784 -0.01614245
  -0.03962736  0.03282262 -0.01430595  0.12822872]
 [-0.00122278  0.03290728  0.04022461 -0.00938234  0.0041625  -0.05285572
  -0.04280645  0.04432166  0.01938271  0.11949807]]
<NDArray 2x10 @gpu(0)>

In [11]:
# Block类是一个通用的部件。事实上，Sequential类继承自Block类
class MySequential(nn.Block):
    def __init__(self, **kwargs):
        super(MySequential, self).__init__(**kwargs)

    def add(self, block):
        # block是一个Block子类实例，假设它有一个独一无二的名字。我们将它保存在Block类的
        # 成员变量_children里，其类型是OrderedDict。当MySequential实例调用
        # initialize函数时，系统会自动对_children里所有成员初始化
        self._children[block.name] = block

    def forward(self, x):
        # Collection.OrderedDict()保证会按照成员添加时的顺序遍历成员
        for block in self._children.values():
            x = block(x)
        return x

In [13]:
net = MySequential()
net.add(nn.Dense(256, activation='relu'))
net.add(nn.Dense(10))
net.initialize(ctx=mx.gpu())
net(X)


[[ 0.0356168  -0.0654178  -0.0182656  -0.03344025 -0.02687218  0.01729953
  -0.07620525 -0.02019735 -0.01474128  0.03277251]
 [ 0.07830474 -0.0699421  -0.04908896 -0.05017216  0.02884687 -0.04290424
  -0.02280661 -0.03931685 -0.00523395  0.05043975]]
<NDArray 2x10 @gpu(0)>

In [47]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 
class FancyMLP(nn.Block):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__(**kwargs)
        # 使用get_constant创建的随机权重参数不会在训练中被迭代（即常数参数）
        self.rand_weight = self.params.get_constant(
            'rand_weight', nd.random.uniform(shape=(20, 20)))
        self.dense = nn.Dense(20, activation='relu')

    def forward(self, x):
        x = self.dense(x)
        # 使用创建的常数参数，以及NDArray的relu函数和dot函数
        x = nd.relu(nd.dot(x, self.rand_weight.data()) + 1)
        # 复用全连接层。等价于两个全连接层共享参数
        x = self.dense(x)
        # 控制流，这里我们需要调用asscalar函数来返回标量进行比较
        while x.norm().asscalar() > 1:
            x /= 2
        if x.norm().asscalar() < 0.8:
            x *= 10
        return x.sum()

In [48]:
net = FancyMLP()
net.initialize(ctx=mx.gpu())
net(X)


[1.3848565]
<NDArray 1 @gpu(0)> 1.3848565

[0.]
<NDArray 1 @gpu(0)>



[29.943895]
<NDArray 1 @gpu(0)>

In [50]:
class NestMLP(nn.Block):

    def __init__(self, **kwargs):
        super(NestMLP, self).__init__(**kwargs)
        self.net = nn.Sequential()
        self.net.add(nn.Dense(64, activation='relu'),
                     nn.Dense(32, activation='relu'))
        self.dense = nn.Dense(16, activation='relu')
       

    def forward(self, x):
        return self.dense(self.net(x))
    


net = nn.Sequential()
net.add(NestMLP(), nn.Dense(20), FancyMLP())

net.initialize(ctx=mx.gpu())
net(X)


[0.86603445]
<NDArray 1 @gpu(0)> 0.86603445

[0.]
<NDArray 1 @gpu(0)>



[2.9067159]
<NDArray 1 @gpu(0)>